In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/Research/onlineCPD

/content/drive/MyDrive/Research/onlineCPD


In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

%matplotlib inline

In [3]:
data_use = "s5"        ### "s1", "s2", "s3", "s4", "s5"
dim_d = 20             ### 10, 20
quantile = 97.5        ### 95, 97.5
quantile_name = 975    ### 95, 975

### Online change point detection procedure

In [ ]:
#########
# NO CP #
#########
#data_C = np.load('data/data_{}_d{}_C.npz'.format(data_use, dim_d))
#print("[INFO] data without CP: ", data_C['y_list'].shape)

In [4]:

class NN(nn.Module):
    def __init__(self, n_in, n_out):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(n_in, 1)

    def forward(self, x):
        x = self.fc1(x).sigmoid()
        return x


def compute_test_stat_nn(X, threshold, t_min=30, n_out_min=10, B=10, delta_max=50, n_epochs=5, LR=0.001, model=NN):

    # Sample size
    n = X.shape[0]

    # Initialization
    T = np.zeros((n, n))

    for t in range(t_min, n):

        n = math.ceil(math.log2(t / 7)) # 7 is a threshold
        binary_cut = []; current_value = t - 3 # 3 is time gap
        binary_cut.append(current_value)
        for i in range(n):
            current_value /= 2
            binary_cut.append(int(math.ceil(current_value)))


        for tau in binary_cut:

            # Initialize neural network
            f = model(n_in=X.shape[1], n_out=1)

            # Parameters of the optimizer
            opt = torch.optim.Adam(f.parameters(), lr=LR)

            X_t = torch.tensor(X[:t, :], dtype=torch.float32, requires_grad=True)

            # weights
            W = torch.cat((torch.ones(tau) * (t - tau), torch.ones(t - tau) * tau)).reshape(-1, 1)

            # Create "virtual" labels
            Y_t = torch.cat((torch.ones(tau), torch.zeros(t - tau))).reshape(-1, 1)

            # Loss function
            loss_fn = nn.BCEWithLogitsLoss(weight=W)

            # Neural network training
            for epoch in range(n_epochs):

                loss = loss_fn(f(X_t), Y_t).mean()
                loss.backward()
                opt.step()
                opt.zero_grad()

            #print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item()}')


            Z = f(X_t).detach().numpy().reshape(-1)

            # Use thresholding to avoid numerical issues
            Z = np.minimum(Z, B)
            Z = np.maximum(Z, -B)

            D = np.zeros(t)
            D[:tau] = 2 / (1 + np.exp(-Z[:tau]))
            D[tau:] = 2 / (1 + np.exp(Z[tau:]))
            D = np.log(D)


            # Compute statistics for each t and each change point candidate tau
            T[tau, t] = tau * (t - tau) / t * (np.mean(D[:tau]) + np.mean(D[tau:]))
            #print("[INFO] T[tau, t]:",T[tau, t])


        '''
        # DETECTION CRITERIA
        if (np.max(T[:, t]) > threshold):
            stopping_time = t
            break
        '''

    # Array of test statistics for each t
    S = np.max(T[:, :], axis=0) # column max for each t

    return S

In [ ]:
'''
# CALIBRATION
threshold_holder = []


for seq_iter in range(100):

    print("[INFO] seq_iter:", seq_iter)
    data = data_C['y_list'][seq_iter]

    # Initialization
    st_nn = 0
    z_nn = 20

    new_S_nn = compute_test_stat_nn(data[st_nn + 1:], z_nn) # statistics for each t from 30 to 100
    #print("[INFO] new_S_nn:", new_S_nn)

    #plt.plot(new_S_nn); plt.xlabel("Index"); plt.ylabel("Value")
    #plt.show()

    threshold_holder.append(max(new_S_nn).item()) # max over t
    print("[INFO] threshold_holder:", threshold_holder)

    df = pd.DataFrame(threshold_holder, columns=['Threshold'])
    df.to_csv('data/threshold_{}_d{}_LogReg.csv'.format(data_use, dim_d), index=False)
'''

In [5]:
###########
# WITH CP #
###########
data_full = np.load('data/data_{}_d{}.npz'.format(data_use, dim_d))
print("[INFO] data with CP: ", data_full['y_list'].shape)


df = pd.read_csv('data/threshold_{}_d{}_LogReg.csv'.format(data_use, dim_d))
#print("[INFO] df.shape", df.shape)
C_by_quantile = np.percentile(df, quantile)
print(f"selected quantile: {C_by_quantile}")

[INFO] data with CP:  (50, 100, 20)
selected quantile: 0.16798373770438352


In [6]:

CP_holder = []


for seq_iter in range(50):

    print("[INFO] seq_iter:", seq_iter)

    data = data_full['y_list'][seq_iter]

    # Initialization
    st_nn = 0
    new_st_nn = 0

    # the threshold
    z_nn = C_by_quantile

    new_S_nn = compute_test_stat_nn(data[st_nn + 1:], z_nn)
    #print("[INFO] new_S_nn:", new_S_nn)

    #plt.plot(new_S_nn)
    #plt.axhline(y=z_nn, color='red', linestyle='--'); plt.xlabel("Index"); plt.ylabel("Value")
    #plt.show()

    indices = np.where(new_S_nn > z_nn)[0]; #print(indices)

    if indices.size > 0:
        CP_holder.append(indices[0].item() + 1)
    else:
        default_value = 99
        CP_holder.append(default_value)

    print(CP_holder)

    df = pd.DataFrame(CP_holder, columns=['CP'])
    df.to_csv('result/{}/CP_{}_d{}_q{}.csv'.format(data_use, data_use, dim_d, quantile_name), index=False)

[INFO] seq_iter: 0
[57]
[INFO] seq_iter: 1
[57, 61]
[INFO] seq_iter: 2
[57, 61, 56]
[INFO] seq_iter: 3
[57, 61, 56, 58]
[INFO] seq_iter: 4
[57, 61, 56, 58, 57]
[INFO] seq_iter: 5
[57, 61, 56, 58, 57, 55]
[INFO] seq_iter: 6
[57, 61, 56, 58, 57, 55, 57]
[INFO] seq_iter: 7
[57, 61, 56, 58, 57, 55, 57, 55]
[INFO] seq_iter: 8
[57, 61, 56, 58, 57, 55, 57, 55, 55]
[INFO] seq_iter: 9
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58]
[INFO] seq_iter: 10
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58]
[INFO] seq_iter: 11
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55]
[INFO] seq_iter: 12
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55, 58]
[INFO] seq_iter: 13
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55, 58, 61]
[INFO] seq_iter: 14
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55, 58, 61, 58]
[INFO] seq_iter: 15
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55, 58, 61, 58, 58]
[INFO] seq_iter: 16
[57, 61, 56, 58, 57, 55, 57, 55, 55, 58, 58, 55, 58, 61, 58, 58, 57]
[INFO] seq_iter: 17
[57, 61, 56, 58, 57, 

In [7]:
def eval(output_t0, Delta, T):
    N = len(output_t0)

    delay_values = [x - Delta for x in output_t0 if x >= Delta]
    delay_sum = sum(delay_values)
    delay_count = len(delay_values)

    pfa_count = sum(1 for x in output_t0 if x < Delta)
    pfn_count = sum(1 for x in output_t0 if x == T)

    delay = delay_sum / delay_count if delay_count != 0 else 0.0
    pfa = pfa_count / N if N != 0 else 0.0
    pfn = pfn_count / N if N != 0 else 0.0

    return delay, pfa, pfn


result = eval(CP_holder, Delta=50, T=99)
print('Eval Metrics:', result)
result_df = pd.DataFrame(result)
file_path = 'result/{}/metric_{}_d{}_q{}.csv'.format(data_use, data_use, dim_d, quantile_name)
result_df.to_csv(file_path, mode='w', header=True, index=False)


Eval Metrics: (6.6, 0.0, 0.0)


# WISDM data set

In [ ]:
# Read the data
df = pd.read_csv('sample_0.csv')
#print(df.shape)

# Feature vectors
X = np.array(df[['X1', 'X2', 'X3']])
print(X.shape)

# Labels
labels = np.array(df['Label'])
change_points = np.where(labels)[0]
print(change_points)

In [ ]:
# Data preprocessing
# Reduce the data, averaging over 20 values

bandwidth = 20
N = X.shape[0] // bandwidth

data = np.empty((0, 3))
new_labels = np.empty(0)

for t in range(N):

    data = np.append(data, np.mean(X[bandwidth * t : bandwidth * (t + 1)], axis=0).reshape(1, -1), axis=0)
    new_labels = np.append(new_labels, np.sum(labels[bandwidth * t : bandwidth * (t + 1)]))

change_points = np.where(new_labels)[0]
print(change_points)

# Add the final moment of time the the list of change points
# (for technical purposes only!)
change_points = np.append(change_points, np.array([data.shape[0]]), axis=0)